# The first QSDC protocol

```Li, Y.B., Wen, Q.Y., Gao, F. et al. Information leak in Liu et al.’s quantum private comparison and a new protocol. Eur. Phys. J. D 66, 110 (2012). https://doi.org/10.1140/epjd/e2012-30065-9```

|Φ±⟩ = 1/√2(|00⟩ ± |11⟩)  
|Ψ±⟩ = 1/√2(|01⟩ ± |10⟩)

1. The message bits are encoded in EPR pairs by using the following rule: 00 → |Φ+⟩, 01 → |Φ−⟩, 10 → |Ψ+⟩ and 11 → |Ψ−⟩.

2. Alice prepares EPR pairs corresponding to her message bits and she takes one qubit from each EPR pair to form a sequence 𝑄𝐴. The remaining partner qubit of each EPR pair forms another sequence 𝑄𝐵 and she sends it to Bob.

3. Bob chooses some qubits of 𝑄𝐵 and measures those qubits randomly in 𝑍 basis (Computational) or 𝑋 basis (Hadamard). Then he tells the positions and the measurement bases of those qubits to Alice, who measures the partner qubits from 𝑄𝐴 in proper bases. They compare the measurement results publicly to check eavesdropping.

4. If there is no eavesdropper, then Alice sends 𝑄𝐴 to Bob and he measures each pair of qubits (one from 𝑄𝐴 and another from 𝑄𝐵) in Bell basis. From the measurement result, Bob gets the message.

5. They choose some random positions of the message bits to check the integrity of the message.

### Assumptions:

1. Channel is noiseless
2. `5. They choose some random positions of the message bits to check the integrity of the message`. This step is not implemented as it is purely classical (and trivial)

In [15]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.tools.visualization import plot_histogram

In [16]:
simulator = Aer.get_backend('qasm_simulator')

In [17]:
def createBellState(M):
    circuit = QuantumCircuit(2, 2)

    if M[0] == "1":
        circuit.x(1)
    if M[1] == "1":
        circuit.x(0)

    circuit.h(0)

    circuit.cx(0, 1)
    return circuit

In [18]:
def performMeasurement(circuit, qubit, basis):
	'''
	basis = C (Computational), H (Hadamard) and B (Bell)
	'''
	if basis == 'C':
			circuit.measure(qubit, qubit)
	if basis == 'H':
			circuit.h(qubit)
			circuit.measure(qubit, qubit)
	if basis == 'B':
			circuit.cx(0, 1)
			circuit.h(0)
			circuit.measure(0, 0)
			circuit.measure(1, 1)
	return circuit

In [19]:
# create a Checking Message

C = ["00", "01", "10", "11", "10"]

### define hadamard measurements and computational measurements

HM = {
    "00": list(sorted(["00", "11"])),
    "01": list(sorted(["01", "10"])),
    "10": list(sorted(["00", "11"])),
    "11": list(sorted(["01", "10"])),
    }
CM = {
    "00": list(sorted(["00", "11"])),
    "01": list(sorted(["00", "11"])),
    "10": list(sorted(["01", "10"])),
    "11": list(sorted(["01", "10"])),
    }

In [20]:
# encode

circuits = [createBellState(b) for b in C]

In [21]:
# send/tamper

T = False # varible to decide if tampering should be performed
# T = True # varible to decide if tampering should be performed

# C = computational
# H = hadamard

tamperBases = ['H', 'C', 'H', 'C', 'H']

if T:
    for i in range(len(circuits)):
        circuits[i] = performMeasurement(circuits[i], 1, tamperBases[i])

In [22]:
# measurement by bob

# C = computational
# H = hadamard

bobBases = ['C', 'C', 'H', 'H', 'H']

for i in range(len(circuits)):
    circuits[i] = performMeasurement(circuits[i], 1, bobBases[i])

In [23]:
# measurement by alice

results = [None]*len(circuits)
counts = [None]*len(circuits)

for i in range(len(circuits)):
    circuits[i] = performMeasurement(circuits[i], 0, bobBases[i])
    results[i] = execute(circuits[i], backend=simulator).result()
    counts[i] = results[i].get_counts(circuits[i])

In [24]:
# verify

total = len(C)
correct = 0

for i in range(len(C)):
    if bobBases[i] == "H":
        if HM[C[i]] == list(sorted(counts[i].keys())):
            correct += 1
    else:
        if CM[C[i]] == list(sorted(counts[i].keys())):
            correct += 1

In [25]:
if correct/total < 0.5:
    print("Eavesdropper present")
else:
    print("Channel is secure")

    # num = input()
    num = 11
    bnum = bin(num)[2:]
    if len(bnum)%2 != 0:
        bnum = "0"+bnum

    for j in range(len(bnum)-4):

        Message = ["00", "00", "00", "10", "11"]

        MessageCircuits = [createBellState(b) for b in Message]

        MessageResults = [None]*len(MessageCircuits)
        MessageCounts = [None]*len(MessageCircuits)

        for i in range(len(MessageCircuits)):
            MessageCircuits[i] = performMeasurement(MessageCircuits[i], -1, "B")
            MessageResults[i] = execute(MessageCircuits[i], backend=simulator).result()
            MessageCounts[i] = MessageResults[i].get_counts(MessageCircuits[i])
        print("Message is:")
        print(" | ".join([list(m.keys())[0] for m in MessageCounts]))

Channel is secure
Message is:
00 | 00 | 00 | 10 | 00


In [26]:
circuits[3].draw(initial_state=True, cregbundle=False)

┌───┐┌───┐     ┌───┐   ┌─┐
q_0: |0>┤ X ├┤ H ├──■──┤ H ├───┤M├
        ├───┤└───┘┌─┴─┐├───┤┌─┐└╥┘
q_1: |0>┤ X ├─────┤ X ├┤ H ├┤M├─╫─
        └───┘     └───┘└───┘└╥┘ ║ 
 c_0: 0 ═════════════════════╬══╩═
                             ║    
 c_1: 0 ═════════════════════╩════

In [27]:
MessageCircuits[3].draw(initial_state=True, cregbundle=False)

┌───┐          ┌───┐┌─┐
q_0: |0>┤ H ├──■────■──┤ H ├┤M├
        ├───┤┌─┴─┐┌─┴─┐└┬─┬┘└╥┘
q_1: |0>┤ X ├┤ X ├┤ X ├─┤M├──╫─
        └───┘└───┘└───┘ └╥┘  ║ 
 c_0: 0 ═════════════════╬═══╩═
                         ║     
 c_1: 0 ═════════════════╩═════